Her er jeres første rigtige opgave! I skal lave bruge [yfinance](https://mercantec.notion.site/yfinance-8f05decbf9a040659da38616e432ed62?pvs=4) også kaldet Yahoo Finance, til at trække data ned på jeres yndlingsaktie! I skal herfra lave en regressionsmodel på dataet. Det involvere at splitte dataet op i træningsdata og testdata. 
Nedenunder er der lige et code-snippet til at få jer startet!

Vi skal bruge følgende pakker til at starte vores projekt, en del af dem kender I. Pandas fra første dag, datetime har i nok arbejdet med lignende pakker før. [yfinance](https://mercantec.notion.site/yfinance-8f05decbf9a040659da38616e432ed62?pvs=4) er den som er linket til ovenfor og har alt det data vi skal bruge. Til sidste er der matplitlib, den er ikke strengt nødvendig for vores projekt, men det hjælper ofte at få et visuelt billede på! Dokumentation kan findes [her](https://mercantec.notion.site/MatPlotLib-bb10bec44c8e4bc28ac511017dbc895d?pvs=4)

In [ ]:
# import yfinance as yf
# import pandas as pd
# from datetime import date, timedelta
# from matplotlib import pyplot as plt

Vi bruger datetime pakken til at sætte et startspunkt og et slutpunkt. Startspunktet er for 365 dage siden og slutpunktet er for 2 dage siden!

In [ ]:
# Start = date.today() - timedelta(365)
# Start.strftime('%Y-%m-%d')

# End = date.today() - timedelta(2)
# End.strftime('%Y-%m-%d')

Vi laver en funktion som tager en "ticker" ind, altså en aktie. 
Her bruger vi så Pandas, pd, til at en dataframe som indeholder vores data omkring den valgte ticker, med slut og startsdato som definere før.

In [ ]:
# def closing_price(ticker):
#     Asset = pd.DataFrame(yf.download(ticker, start=Start,
#       end=End))
#     return Asset

Nu kan vi bruge vores funktion til at se på vores data. Her bruger vi Microsoft aktien, MSFT. Standarten er at vi for dato, åbningsprisen, højeste og laveste på dagen, lukke prisen og den justerede lukkepris. Vi kan så bruge matplotlib

In [ ]:
# microsoft = closing_price('MSFT')
# print(microsoft)
# plt.plot(microsoft['Adj Close'])
# plt.show()

# Opgaven

Opgaven går nu på at I skal bruge jeres viden omkring regression på at lave en model som kan forudsige hvad aktie prisen bliver over den næste periode eller forhåbenligt bare en retning. 

Det er ikke målet at I skal finde en model som gør det, men at I skal lege rundt med det. De værktøjer som vi har fået indtil videre, er ikke nok til at kunne lave en model som har noget grundlag. Det data vi for er heller ikke nok. 

I kan tage inspiration ud fra dokumenterne her [regression](https://github.com/MAGS-Template/MachineLearning/blob/main/1.%20Regression/1.%20regression.ipynb) og [regression med data](https://github.com/MAGS-Template/MachineLearning/blob/main/1.%20Regression/2.%20regression_with_data.ipynb) - det er filerne i samme mappe som den her!


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

Start = date.today() - timedelta(1095)
Start.strftime('%Y-%m-%d')
End = date.today() - timedelta(2)
End.strftime('%Y-%m-%d')

In [ ]:
# Functions
def logger(data, extra = "\n"):
    if (True):
        print(data, extra)

def closing_price(ticker):
    Asset = pd.DataFrame(yf.download(ticker, start=Start,
      end=End))
    return Asset

def build_and_compile_model(input_shape):
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(units=64,
                                return_sequences=True,
                                input_shape=input_shape))
    model.add(keras.layers.LSTM(units=64))
    model.add(keras.layers.Dense(32))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1))
    model.compile(loss='mean_squared_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

    return model

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# getting the dataset and cleaning
# stockData = closing_price('MSFT')
# stockData = closing_price('AAPL')
# stockData = closing_price('TSLA')
stockData = closing_price('GOOG')
stockData.drop(columns=['Adj Close'], inplace=True)
dataset = stockData.copy()
dataset = dataset.dropna()

# normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)
scaled_data = pd.DataFrame(scaled_data, index=dataset.index, columns=dataset.columns)
logger(scaled_data)

In [ ]:
# Splitting the data into a training set and a test set.
# train_dataset = scaled_data.sample(frac=0.8, random_state=0)
# test_dataset = scaled_data.drop(train_dataset.index)
train_dataset, test_dataset = np.split(scaled_data, [int(.8*len(scaled_data))])

print(train_dataset.head())
print(test_dataset.head())


print("training dataset count:")
print(train_dataset.count(), '\n')
print("test dataset count:")
print(test_dataset.count())

In [ ]:
train_dataset.describe().transpose()

In [ ]:
# Split data into features and labels
train_features = train_dataset.copy().astype('float32')
test_features = test_dataset.copy().astype('float32')

train_labels = train_features.pop('Close')
test_labels = test_features.pop('Close')

In [ ]:
logger(train_features.shape[1])
dnn_model = build_and_compile_model((train_features.shape[1],1))
logger(dnn_model.summary())

In [ ]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=.02,
    epochs=100)

In [ ]:
test_results = {}
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)
logger(test_results)

In [ ]:
# dnn_model = tf.keras.models.load_model("models/MSFT")
#dnn_model.save('models/MSFT')
test_predictions = dnn_model.predict(test_features).flatten()


logger(len(test_features))
logger(len(test_labels))
logger(test_labels.head())
test_predictions = pd.Series(test_predictions, index=test_labels.index)

plt.figure(1)
plt.plot(train_labels, color="b")
plt.plot(test_predictions, color="r")
plt.plot(test_labels, color="g")
plt.show()
logger(test_labels.compare(test_predictions).tail())